In [1]:
import numpy as np
import pyopencl as cl

In [2]:
a_np = (np.random.rand(50000).astype(np.float32)) * 2
b_np = (np.random.rand(50000).astype(np.float32)) * 2


In [3]:
ctx = cl.create_some_context()

In [4]:
queue = cl.CommandQueue(ctx)

In [5]:
mf = cl.mem_flags
a_g = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=a_np)
b_g = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=b_np)

In [6]:
prg = cl.Program(ctx, """
__kernel void sum(__global const float *a_g, __global const float *b_g, __global float *res_g) {
  int gid = get_global_id(0);
  res_g[gid] = a_g[gid] + b_g[gid];
}
""").build()


In [7]:
res_g = cl.Buffer(ctx, mf.WRITE_ONLY, a_np.nbytes)
prg.sum(queue, a_np.shape, None, a_g, b_g, res_g)


In [8]:
res_np = np.empty_like(a_np)
cl.enqueue_copy(queue, res_np, res_g)

In [9]:
# Check on CPU with Numpy:
print(res_np - (a_np + b_np))
print(np.linalg.norm(res_np - (a_np + b_np)))

[ 0.  0.  0. ...,  0.  0.  0.]
0.0


In [ ]:
a_np

In [ ]:
plat = cl.get_platforms()

In [ ]:
plat

In [ ]:
import reikna

In [10]:
%matplotlib inline
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt

import numpy as np

import scipy.signal as sps

from IPython.display import HTML
import IPython.display 
from io import BytesIO
from base64 import b64encode
from PIL import Image

In [11]:
import matplotlib.pylab as pylab
pylab.rcParams['figure.figsize'] = 32, 24

In [12]:
infd = open('rotplane.ld', 'rb')
#infd = open('ve-zone.ld', 'rb')

bufsize = 1820 * 525 * 30
inbuf = infd.read(bufsize * 2)
data = np.fromstring(inbuf, 'uint16', len(inbuf)//2)
data_orig = data.copy()

print(data.shape)

(11475882,)


In [13]:
# Draws a uint16 image as a uint8, defaults to one frame
def drawdata(bm, x = 1820, y = 525, hscale = 1, vscale = 1, outsize = None):
#    bmf = np.float32(bm) / 65536.0
    if y is None:
        y = len(bm) // x
        
    if outsize is None:
        outsize = (x * hscale, y * vscale)
    
    bmf = np.uint8(bm[0:x*y] / 256.0)
    print(bmf.shape)
    if x is not None:
        bms = (bmf.reshape(len(bmf)//x, -1))
    else:
        bms = bmf
    
    print(bms.dtype, bms.shape, bms[:][0:y].shape)
    im = Image.fromarray(bms[0:y])
    im = im.resize(outsize)
#    imshow(np.asarray(im))
    b = BytesIO()
    im.save(b, format='png')
    return IPython.display.Image(b.getvalue())
                         
#drawdata(data)                  

In [14]:
# This follows the default scale in lddecodercuda
minire = -60
maxire = 140

hz_ire_scale = (9300000 - 8100000) / 100
minn = 8100000 + (hz_ire_scale * -60)

out_scale = 65534.0 / (maxire - minire)

def RawToIRE(data):
    return (np.float32(data) / out_scale) + minire

def IREToRaw(data):
    return np.uint16((data - minire) * out_scale)

In [15]:
blocklen = 1024 * 1024

def filtfft(filt, blen = blocklen):
    return sps.freqz(filt[0], filt[1], blen, whole=1)[1]

In [115]:
from reikna.cluda import dtypes, ocl_api
from reikna.fft import FFT
from reikna.core import Annotation, Type, Transformation, Parameter

api = ocl_api()

#thr = api.Thread.create()
thr = api.Thread(ctx)

In [35]:
FSC = 8
freq_mhz = (315.0 / 88.0) * FSC
freq_hz = freq_mhz * 1000000.0

#Ncolor = 24
#sync_filter = sps.firwin(Ncolor + 1, 0.1 / (freq_mhz / 2.0), window='hamming')



In [116]:
def findpeaks(data):
    dinput = np.diff(data)
    dpeaks = (dinput[:-1] > 0) & (dinput[1:] < 0)
    
    return np.where(dpeaks)[0] 
    
f_syncid = sps.butter(3, 0.002)
f_syncid_offset = 320

Fsync = filtfft(f_syncid)

Fsync_gpu = thr.to_device(Fsync)

#sfdata = None

In [117]:
f_slpf_b = sps.firwin(49, 100000/freq_hz)
f_slpf_a = [1.0]
f_slpf_offset = 24

In [118]:
# from https://github.com/fjarri/reikna/blob/develop/examples/demo_real_to_complex_fft.py

# A transformation that transforms a real array to a complex one
# by adding a zero imaginary part
def get_complex_trf(arr):
    complex_dtype = dtypes.complex_for(np.float32)
    return Transformation(
        [Parameter('output', Annotation(Type(complex_dtype, arr.shape), 'o')),
        Parameter('input', Annotation(arr, 'i'))],
        """
        ${output.store_same}(
            COMPLEX_CTR(${output.ctype})(
                ${input.load_same},
                0));
        """)


In [119]:
#fftfood = np.float32(data[0:blocklen])
fftfood = data[0:blocklen]
trf = get_complex_trf(fftfood)

In [120]:
# Create the FFT computation and attach the transformation above to its input.
fft = FFT(trf.output) # (A shortcut: using the array type saved in the transformation)
fft.parameter.input.connect(trf, trf.output, new_input=trf.input)
cfft = fft.compile(thr)

ffti = FFT(trf.output)
cffti = ffti.compile(thr)

In [166]:
import time
t1 = time.time()
# Run the computation
arr_dev = thr.to_device(fftfood)
res_dev = thr.array(fftfood.shape, np.complex64)
res2_dev = thr.array(fftfood.shape, np.complex64)

t2 = time.time()

cfft(res_dev, arr_dev.data)
res_dev *= Fsync_gpu
#cffti(res2_dev, res_dev, 1)
cffti(res2_dev, res_dev, 1)

t3 = time.time()

#result = res_dev.get()
result2 = res2_dev.get()

t4 = time.time()

print(t2 - t1, t3 - t1, t4 - t1, 1 / (t4 - t1))

0.0024824142456054688 0.004161357879638672 0.007703065872192383 129.81844068216287


In [ ]:
result

In [ ]:
offset = 330
plt.plot(np.abs(result2)[offset+1000:offset+3500])
plt.plot(fftfood[1000:3500])

In [ ]:
fftfood[0:10]

In [ ]:
res_dev

In [ ]:
Fsync

In [ ]:
type(res_dev)

In [ ]:
res_dev.data

In [ ]:
Fsync_gpu = thr.to_device(Fsync)

In [ ]:
import time

In [ ]:
time.time()

In [89]:
type(arr_dev)

reikna.cluda.ocl.Array

In [90]:
arr = arr_dev.data

In [91]:
arr?

In [92]:
import reikna.cluda.ocl

AttributeError: 'Buffer' object has no attribute '_queue'

In [102]:
arr_dev.data

In [109]:
res_g = cl.Buffer(ctx_a, mf.WRITE_ONLY, 1024*1024*8)

In [104]:
thr

In [107]:
ctx_a = arr_dev.data.context

In [112]:
thr

AttributeError: 'Context' object has no attribute 'device_params'

In [114]:
thr_1 = api.Thread(ctx)